In [1]:
import datetime

import numpy as np
import pandas as pd
import yaml
from sqlalchemy import create_engine


# database connections 

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_co = config['CO_SA']
    config_etl = config['ETL_PRO']

# Construct the database URL
url_co = (f"{config_co['drivername']}://{config_co['user']}:{config_co['password']}@{config_co['host']}:"
          f"{config_co['port']}/{config_co['dbname']}")
url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@{config_etl['host']}:"
           f"{config_etl['port']}/{config_etl['dbname']}")
# Create the SQLAlchemy Engine
co_sa = create_engine(url_co)
etl_conn = create_engine(url_etl)

# Extract

In [3]:
df_trans = pd.read_sql_table('trans_servicio', etl_conn)
dim_persona = pd.read_sql_table('dim_persona', etl_conn)
dim_medico = pd.read_sql_table('dim_medico', etl_conn)
dim_servicio = pd.read_sql_table('dim_servicio', etl_conn)
dim_ips = pd.read_sql_table('dim_ips', etl_conn)
dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)

ValueError: Table dim_servicio not found

# Transformations

In [ ]:
hecho_atencion = pd.merge(df_trans,dim_fecha[['date','key_dim_fecha']],left_on='fecha_atencion',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_dim_fecha':'key_fecha_atencion','id_medico':'cedula','id_usuario':'numero_identificacion'}, inplace=True)
hecho_atencion = pd.merge(hecho_atencion,dim_fecha[['date','key_dim_fecha']],left_on='fecha_solicitud',right_on='date')
hecho_atencion.drop(columns=['date'], inplace=True)
hecho_atencion.rename(columns={'key_dim_fecha':'key_fecha_solicitud'}, inplace=True)

hecho_atencion.head()

In [ ]:
hecho_atencion= hecho_atencion.merge(dim_persona[['key_dim_persona','numero_identificacion']])
hecho_atencion.drop(columns=['numero_identificacion'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_medico[['key_dim_medico','cedula','id_ips']])
hecho_atencion.drop(columns=['cedula'], inplace=True)

hecho_atencion= hecho_atencion.merge(dim_ips[['key_dim_ips','id_ips']])
hecho_atencion.drop(columns=['id_ips'], inplace=True)
hecho_atencion.head()

In [ ]:
hecho_atencion= hecho_atencion.merge(dim_servicio[['name','key_dim_servicio']],left_on='tipo_servicio',right_on='name')
hecho_atencion.drop(columns=['name','tipo_servicio'], inplace=True)
hecho_atencion.head()

# tiempo de espera

In [ ]:
hecho_atencion['tiempo_espera'] = hecho_atencion['fecha_hora_atencion'] - hecho_atencion['fecha_hora_solicitud']
hecho_atencion['tiempo_espera_dias'] = hecho_atencion['tiempo_espera'].dt.days
hecho_atencion['tiempo_espera_minutos'] = hecho_atencion['tiempo_espera'].dt.seconds // 60 
hecho_atencion['tiempo_espera_horas'] = hecho_atencion['tiempo_espera'].dt.seconds // (60 * 60)
hecho_atencion['tiempo_espera_segundos'] = hecho_atencion['tiempo_espera'].dt.seconds

hecho_atencion.drop(columns=['tiempo_espera','fecha_atencion','fecha_solicitud','hora_solicitud','hora_atencion','fecha_hora_solicitud','fecha_hora_atencion','codigo_servicio'], inplace=True)
hecho_atencion.head()


In [ ]:
from datetime import date

hecho_atencion['saved'] = date.today()

# load

In [ ]:
hecho_atencion.to_sql('hecho_atencion',etl_conn,if_exists='replace',index_label='key_hecho_atencion')